In [1]:
import pandas as pd
import numpy as np
import json
from PIL import Image, ImageDraw, ImageFont
import os
import regex

In [2]:
def preprocess_text(text):
    text = text.replace('\'\'', '').lower()
    split_text = regex.findall(r'\':?(.*?):?\'', text)
    return split_text

def split_emojis(s):
    return regex.findall(r"\X", s)

def preprocess_emoji(emojis):
    emojis_list = split_emojis(emojis)
    filtered = [c for c in emojis_list if c.strip() != '']
    return filtered

elco_df = pd.read_csv('../../data/elco.csv')
descs = elco_df["Description"]
emoji_dict = {}

for i, row in elco_df.iterrows():
    descs = preprocess_text(row["Description"])
    emojis = preprocess_emoji(row["EM"])
    if len(descs) != len(emojis):
        print(f"Error at: {i}, {descs}, {emojis}")
        break
    else:
        for j in range(len(descs)):
            emoji_dict[descs[j]] = emojis[j]
            
emoji_desc_pair = {pair[0]: pair[1] for pair in set(emoji_dict.items())}

In [3]:
def emoji_to_image(em, desc, font_path, image_size=128, save_dir='images'):
    os.makedirs(save_dir, exist_ok=True)
    image = Image.new("RGB", (image_size, image_size), (255, 255, 255))
    draw = ImageDraw.Draw(image)

    font = ImageFont.truetype(font_path, 109)
    draw.text((0, 0), em, font=font, embedded_color=True)
    image.save(os.path.join(save_dir, f"{desc}.png"))


In [17]:
import math
def combine_emoji_images(file_path_list, output_size=None):
    images = [Image.open(p).resize((128, 128)) for p in file_path_list]
    num_images = len(images)

    cols = math.ceil(math.sqrt(num_images))
    rows = math.ceil(num_images / cols)

    if output_size:
        total_width, total_height = output_size
        cell_width = total_width // cols
        cell_height = total_height // rows
        images = [img.resize((cell_width, cell_height)) for img in images]
    else:
        cell_width, cell_height = 128, 128
        total_width = cell_width * cols
        total_height = cell_height * rows

    combined_img = Image.new("RGB", (total_width, total_height), (255, 255, 255))

    for idx, img in enumerate(images):
        x = (idx % cols) * cell_width
        y = (idx // cols) * cell_height
        combined_img.paste(img, (x, y))

    return combined_img

In [109]:
for desc, em in emoji_desc_pair.items():
    emoji_to_image(em, desc, '../../NotoColorEmoji.ttf')

In [9]:
from tqdm import tqdm

In [18]:
for i, row in tqdm(elco_df.iterrows(), total=elco_df.shape[0]):
    descs = preprocess_text(row["Description"])
    emojis = preprocess_emoji(row["EM"])
    if len(descs) != len(emojis):
        print(f"Error at: {i}, {descs}, {emojis}")
        break
    emoji_paths = [os.path.join('../../images', f"{desc}.png") for desc in descs]
    combined_image = combine_emoji_images(emoji_paths, output_size=(512, 512))
    os.makedirs('combine_em_images', exist_ok=True)
    combined_image.save(os.path.join('combine_em_images', f"{i}.png"))

100%|██████████| 1655/1655 [00:52<00:00, 31.69it/s]
